In [3]:
import requests
from datetime import datetime, timedelta

api_key = "ljmJ6y8jH7XpdY1iJ68nADTczTfPhVDS"

# Generate start and end dates for the next 10 days
start_date = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
end_date = (datetime.now() + timedelta(days=14)).strftime("%Y-%m-%dT%H:%M:%SZ")

# API request with U.S. filter, price filter, and 14-day range
url = f"https://app.ticketmaster.com/discovery/v2/events.json?classificationName=music&countryCode=US&startDateTime={start_date}&endDateTime={end_date}&apikey={api_key}"
url += "&size=200&page=10"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print("total events ", data["page"]["totalElements"])
    if "_embedded" in data:
        print(f"\n=== Data Accessed At: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} ===\n")
        print("how many events in page: ", len(data["_embedded"]["events"]))
        for event in data["_embedded"]["events"]:
            
            # Filter: Only process events where priceRanges is available
            if 'priceRanges' in event:
                print("=== Event Details ===")
                print(f"Event Name: {event['name']}")
                print(f"Location: {event['_embedded']['venues'][0]['city']['name']}, {event['_embedded']['venues'][0]['state']['stateCode']}")
                print(f"Event ID: {event['id']}")
                print(f"Start Date: {event['dates']['start']['localDate']}")

                # Handle price ranges or single prices
                for price in event['priceRanges']:
                    min_price = price.get('min')
                    max_price = price.get('max')
                    currency = price.get('currency', 'Unknown')

                    # Display single price or range
                    if min_price and max_price and min_price != max_price:
                        print(f"Price Range: {min_price} {currency} - {max_price} {currency}")
                    else:
                        single_price = min_price if min_price else max_price
                        print(f"Price: {single_price} {currency}" if single_price else "Price: Not available")

                # Ticket limit
                ticket_limit = event.get('ticketLimit', {}).get('info', 'No limit info available')
                print(f"Ticket Limit: {ticket_limit}")

                # Classifications (genre, subgenre)
                if 'classifications' in event:
                    classification = event['classifications'][0]
                    genre = classification.get('genre', {}).get('name', 'N/A')
                    sub_genre = classification.get('subGenre', {}).get('name', 'N/A')
                    event_type = classification.get('type', {}).get('name', 'N/A')
                    print(f"Genre: {genre}, Sub-Genre: {sub_genre}")

                # Venue information
                venue = event['_embedded']['venues'][0]
                print("\n=== Venue Details ===")
                print(f"Venue Name: {venue['name']}")
                print(f"Venue ID: {venue['id']}")
                print(f"Address: {venue['address'].get('line1', 'N/A')}")
                print(f"City: {venue['city']['name']}")
                print(f"State: {venue['state']['stateCode']}")
                
                print(f"Ticket URL: {event['url']}\n")
    else:
        print("No events found within the specified range.")
else:
    print("Error:", response.status_code, response.text)


Error: 400 {"errors":[{"code":"DIS1035","detail":"API Limits Exceeded: Max paging depth exceeded. (page * size) must be less than 1,000","status":"400","_links":{"about":{"href":"/discovery/v2/errors.html#DIS1035"}}}]}
